In [1]:
import numpy as np
import pandas as pd
import pandasql as psql

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier


from imblearn.over_sampling import SMOTE

/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [2]:
df = pd.read_csv("~/Downloads/bigml_59c28831336c6604c800002a.csv")

In [3]:
le = LabelEncoder()
df['state'] = le.fit_transform(df['state'])
df['international plan'] = le.fit_transform(df['international plan'])
df['voice mail plan'] = le.fit_transform(df['voice mail plan'])
df['area code'] = le.fit_transform(df['area code'])

In [4]:
df.describe()

,state,account length,area code,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls
count,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000
mean,26.059406,101.064806,1.000600,0.096910,0.276628,8.099010,179.775098,100.435644,30.562307,200.980348,100.114311,17.083540,200.872037,100.107711,9.039325,10.237294,4.479448,2.764581,1.562856
std,14.824911,39.822106,0.709649,0.295879,0.447398,13.688365,54.467389,20.069084,9.259435,50.713844,19.922625,4.310668,50.573847,19.568609,2.275873,2.791840,2.461214,0.753773,1.315491
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,23.200000,33.000000,1.040000,0.000000,0.000000,0.000000,0.000000
25%,14.000000,74.000000,0.000000,0.000000,0.000000,0.000000,143.700000,87.000000,24.430000,166.600000,87.000000,14.160000,167.000000,87.000000,7.520000,8.500000,3.000000,2.300000,1.000000
50%,26.000000,101.000000,1.000000,0.000000,0.000000,0.000000,179.400000,101.000000,30.500000,201.400000,100.000000,17.120000,201.200000,100.000000,9.050000,10.300000,4.000000,2.780000,1.000000
75%,39.000000,127.000000,2.000000,0.000000,1.000000,20.000000,216.400000,114.000000,36.790000,235.300000,114.000000,20.000000,235.300000,113.000000,10.590000,12.100000,6.000000,3.270000,2.000000
max,50.000000,243.000000,2.000000,1.000000,1.000000,51.000000,350.800000,165.000000,59.640000,363.700000,170.000000,30.910000,395.000000,175.000000,17.770000,20.000000,20.000000,5.400000,9.000000


In [5]:
df.dtypes

state                       int64
account length              int64
area code                   int64
phone number               object
international plan          int64
voice mail plan             int64
number vmail messages       int64
total day minutes         float64
total day calls             int64
total day charge          float64
total eve minutes         float64
total eve calls             int64
total eve charge          float64
total night minutes       float64
total night calls           int64
total night charge        float64
total intl minutes        float64
total intl calls            int64
total intl charge         float64
customer service calls      int64
churn                        bool
dtype: object

In [6]:
q1 = ("""
SELECT *,
CASE 
    WHEN "total intl calls" > 0 THEN 1
    ELSE 0 
END AS intlcalls,
CASE 
    WHEN "total eve calls" > 0 THEN 1
    ELSE 0 
END AS evecalls,
CASE 
    WHEN "total night calls" > 0 THEN 1
    ELSE 0
END AS nightcalls,
CASE
    WHEN "total day calls" > 0 THEN 1
    ELSE 0
END AS daycalls,
CASE
    WHEN "customer service calls" > 1 THEN 1 
    ELSE 0
END AS customerservice
FROM df
""")
df1 = psql.sqldf(q1)
df1 = df1.drop(columns = ['total intl calls','total eve calls','total night calls','total day calls','customer service calls'])

In [7]:
q2 = """
SELECT "state", "area code",
"international plan", "voice mail plan", "number vmail messages"/"account length",
"total day minutes"/"account length", "total day calls"/"account length", "total day charge"/"account length",
"total eve minutes"/"account length", "total eve calls"/"account length", "total eve charge"/"account length",
"total night minutes"/"account length", "total night calls"/"account length", "total night charge",
"total intl minutes"/"account length", "total intl calls"/"account length", "total intl charge"/"account length",
"customer service calls", "churn"
FROM df
"""
df2 = psql.sqldf(q2)

In [8]:
# actually we dont really need the column phone number because it will cause over fitting and will not have a 
# correlation between our train and test data.

In [9]:
X = df.drop(columns = ['churn','phone number'])
y = df['churn']
Xtrain,Xtest,ytrain,ytest = train_test_split(X,y,random_state=42)

In [10]:
dtc = DecisionTreeClassifier(max_depth=22)
dtc.fit(Xtrain,ytrain)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=22,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [11]:
y_preds = dtc.predict(Xtrain)

In [12]:
confusion_matrix(ytrain,y_preds)

array([[2141,    0],
       [   0,  358]])

In [13]:
X = df.drop(columns = ['churn','phone number'])
y = df['churn']
Xtrain,Xtest,ytrain,ytest = train_test_split(X,y,random_state=42)

In [14]:
rfc = RandomForestClassifier(n_estimators=100,random_state=42)

In [15]:
rfc.fit(Xtrain,ytrain)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [16]:
confusion_matrix(ytrain,rfc.predict(Xtrain))

array([[2141,    0],
       [   0,  358]])

In [17]:
rfc.feature_importances_

array([0.02472057, 0.02867274, 0.00857789, 0.08677414, 0.02023758,
       0.02702115, 0.13428955, 0.03074894, 0.13058729, 0.06298392,
       0.02874081, 0.06677738, 0.03676844, 0.03198688, 0.03508086,
       0.04161329, 0.0448703 , 0.03693667, 0.1226116 ])

In [18]:
print(classification_report(ytest,rfc.predict(Xtest)))

              precision    recall  f1-score   support

       False       0.95      1.00      0.97       709
        True       0.97      0.68      0.80       125

    accuracy                           0.95       834
   macro avg       0.96      0.84      0.88       834
weighted avg       0.95      0.95      0.94       834



In [19]:
rfc.fit(Xtrain,ytrain)
confusion_matrix(ytest,rfc.predict(Xtest))

array([[706,   3],
       [ 40,  85]])

In [20]:
sm = SMOTE(random_state=42, sampling_strategy=1.0)
Xsmote, ysmote = sm.fit_sample(Xtrain, ytrain)
rfc.fit(Xsmote,ysmote)
confusion_matrix(ytest,rfc.predict(Xtest))

array([[685,  24],
       [ 26,  99]])

In [21]:
print(classification_report(ytest,rfc.predict(Xtest)))

              precision    recall  f1-score   support

       False       0.96      0.97      0.96       709
        True       0.80      0.79      0.80       125

    accuracy                           0.94       834
   macro avg       0.88      0.88      0.88       834
weighted avg       0.94      0.94      0.94       834



In [25]:
X2 = df2.drop(columns = ['churn'])
y2 = df2['churn']
Xtrain2,Xtest2,ytrain2,ytest2 = train_test_split(X2,y2,random_state=42)

In [26]:
rfc.fit(Xtrain2,ytrain2)
y_preds2 = rfc.predict(Xtrain2)
confusion_matrix(ytrain2,y_preds2)

array([[2141,    0],
       [   0,  358]])

In [27]:
print(classification_report(ytest2,rfc.predict(Xtest2)))

              precision    recall  f1-score   support

           0       0.89      0.98      0.93       709
           1       0.75      0.31      0.44       125

    accuracy                           0.88       834
   macro avg       0.82      0.65      0.69       834
weighted avg       0.87      0.88      0.86       834



In [28]:
confusion_matrix(ytest2,rfc.predict(Xtest2))

array([[696,  13],
       [ 86,  39]])

In [29]:
sm = SMOTE(random_state=42, sampling_strategy=1.0)
Xsmote1, ysmote1 = sm.fit_sample(Xtrain2, ytrain2)

In [30]:
rfc.fit(Xsmote1,ysmote1)
confusion_matrix(ytest2,rfc.predict(Xtest2))

array([[663,  46],
       [ 67,  58]])

In [31]:
print(classification_report(ytest2,rfc.predict(Xtest2)))

              precision    recall  f1-score   support

           0       0.91      0.94      0.92       709
           1       0.56      0.46      0.51       125

    accuracy                           0.86       834
   macro avg       0.73      0.70      0.71       834
weighted avg       0.86      0.86      0.86       834



In [118]:
df3['account months'] = np.ceil(df['account length']/30).astype(int)

In [54]:
# q3 = ("""
# SELECT *,
# CASE 
#     WHEN "account length" < 30 THEN 0
#     WHEN "account length" BETWEEN 31 and 60 THEN 1
#     WHEN "account length" BETWEEN 61 and 90 THEN 2
#     ELSE 3
     
# END AS gtr_avg
# FROM df
# """)
# df3 = psql.sqldf(q3)
# df3 = df3.drop(columns = ['phone number','account length'])

In [127]:
X = df3.drop(columns = ['churn'])
y = df3['churn']
Xtrain,Xtest,ytrain,ytest = train_test_split(X,y,random_state=42)
rfc.fit(Xtrain,ytrain)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [128]:
confusion_matrix(ytest,rfc.predict(Xtest))

array([[709,   0],
       [ 22, 103]])

In [129]:
print(classification_report(ytest,rfc.predict(Xtest)))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98       709
           1       1.00      0.82      0.90       125

    accuracy                           0.97       834
   macro avg       0.98      0.91      0.94       834
weighted avg       0.97      0.97      0.97       834



In [122]:
sm = SMOTE(random_state=42, sampling_strategy=1)
Xsmote, ysmote = sm.fit_sample(Xtrain, ytrain)
rfc.fit(Xsmote,ysmote)
confusion_matrix(ytest,rfc.predict(Xtest))

array([[705,   4],
       [ 16, 109]])

In [123]:
print(classification_report(ytest,rfc.predict(Xtest)))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99       709
           1       0.96      0.87      0.92       125

    accuracy                           0.98       834
   macro avg       0.97      0.93      0.95       834
weighted avg       0.98      0.98      0.98       834



In [130]:
rfc.feature_importances_

array([0.01781716, 0.00638473, 0.0700475 , 0.04050099, 0.03527827,
       0.07077254, 0.02088462, 0.06580524, 0.02893321, 0.02063612,
       0.02988229, 0.02243889, 0.0209801 , 0.0227309 , 0.03204692,
       0.03310242, 0.03132106, 0.12506148, 0.00436784, 0.06827159,
       0.02174009, 0.17881049, 0.02398501, 0.00820054])

In [131]:
list(zip(Xtest.columns,rfc.feature_importances_))

[('state', 0.017817162799637905),
 ('area code', 0.006384731102192434),
 ('international plan', 0.07004750183899447),
 ('voice mail plan', 0.0405009942113875),
 ('number vmail messages', 0.0352782720852878),
 ('total day minutes', 0.07077253632098861),
 ('total day calls', 0.020884619897356557),
 ('total day charge', 0.06580524014527062),
 ('total eve minutes', 0.02893320747055677),
 ('total eve calls', 0.020636122510368043),
 ('total eve charge', 0.029882288383734898),
 ('total night minutes', 0.022438894148957878),
 ('total night calls', 0.02098009994651926),
 ('total night charge', 0.022730899612873647),
 ('total intl minutes', 0.032046915351350035),
 ('total intl calls', 0.03310242091807169),
 ('total intl charge', 0.03132105817440855),
 ('customer service calls', 0.12506148072178142),
 ('gtr_avg', 0.004367835752418694),
 ('total minutes', 0.06827158586934881),
 ('total calls', 0.021740087940116375),
 ('total cost', 0.17881049016187323),
 ('cost per day', 0.02398500963866582),
 ('a

In [124]:
Xtest.describe()

,state,area code,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,...,total intl calls,total intl charge,customer service calls,gtr_avg,account length,total minutes,total calls,total cost,cost per day,account months
count,834.000000,834.000000,834.000000,834.000000,834.000000,834.000000,834.000000,834.000000,834.000000,834.000000,...,834.000000,834.000000,834.000000,834.000000,834.000000,834.000000,834.000000,834.000000,834.000000,834.000000
mean,25.785372,1.032374,0.097122,0.290168,8.438849,179.083573,100.425659,30.444880,201.305396,100.621103,...,4.576739,2.748094,1.545564,2.432854,3.902878,592.189329,306.342926,59.377374,0.991785,3.902878
std,14.887113,0.715231,0.296302,0.454112,13.774700,55.705695,20.398959,9.469929,50.997535,19.465495,...,2.547411,0.749223,1.274352,0.837836,1.381203,91.433173,34.869882,10.797792,3.383636,1.381203
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,31.200000,48.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,301.500000,206.000000,22.930000,0.174791,1.000000
25%,13.250000,1.000000,0.000000,0.000000,0.000000,143.250000,86.000000,24.350000,167.300000,87.000000,...,3.000000,2.270000,1.000000,2.000000,3.000000,526.350000,284.000000,52.100000,0.444992,3.000000
50%,26.500000,1.000000,0.000000,0.000000,0.000000,180.450000,101.000000,30.680000,202.600000,101.000000,...,4.000000,2.750000,1.000000,3.000000,4.000000,595.750000,306.000000,59.370000,0.575734,4.000000
75%,38.000000,2.000000,0.000000,1.000000,21.000000,216.525000,114.750000,36.807500,236.925000,114.000000,...,6.000000,3.285000,2.000000,3.000000,5.000000,655.325000,329.000000,66.897500,0.799857,5.000000
max,50.000000,2.000000,1.000000,1.000000,46.000000,335.500000,163.000000,57.040000,341.300000,164.000000,...,20.000000,5.100000,7.000000,3.000000,9.000000,882.200000,416.000000,90.460000,69.950000,9.000000


In [ ]:
# make a total time category

In [73]:
df3['total minutes'] = df['total day minutes']+df['total eve minutes']+df['total night minutes']+df['total intl minutes']
df3['total calls'] = df['total day calls']+df['total eve calls']+df['total night calls']+df['total intl calls']
df3['total cost'] = df['total day charge']+df['total eve charge']+df['total night charge']+df['total intl charge']

In [81]:
df3['cost per day'] = df3['total cost']/df['account length']